Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Hyper-parameter tuning using Azure ML service

The hyper-parameters of methods presented in this tutorial are tuned using Hyperdrive, a feature of Azure Machine Learning (Azure ML) service. Before running this notebook, please follow instructions in [configuration notebook](../configuration.ipynb) and provision Azure ML workspace.

The running time depends on the size of your Azure ML cluster and the method being tuned.

This notebook is used to tune several approaches:
- Feed-forward network multi-step multivariate approach (`ff_multistep_config.json`)
- RNN multi-step approach (`rnn_multistep_config.json`)
- RNN teacher forcing approach (`rnn_teacher_forcing_config.json`)
- RNN encoder decoder approach (`rnn_encoder_decoder_config.json`)

Each of these use cases is defined in a json configuration file listed above in parenthesis. To run a specific approach, please specify the appropriate configuration file in the next section

## Import required libraries

In [38]:
import json
import azureml
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.dnn import TensorFlow
from azureml.train.hyperdrive import (
    RandomParameterSampling,
    HyperDriveConfig,
    PrimaryMetricGoal,
    choice,
)

## Specify and import configuration file

Please specify the configuration file for this notebook run. Configuration file is a json file containing the following parameters that will be used by the notebook:
- EXPERIMENT_NAME
- CLUSTER_NAME
- SCRIPT_DIR
- SCRIPT_NAME
- SCRIPT_PARAMS
- HYPER_PARAMS
- MODEL_NAME

In [14]:
config_file = "ff_multistep_config.json"

try:
    with open(config_file) as f:
        params = json.load(f)
except IOError:
    print("Config file not found.")

In [15]:
print("Specified notebook parameters:\n\n", params)

Specified notebook parameters:

 {'EXPERIMENT_NAME': 'ff-multistep', 'CLUSTER_NAME': 'gpucluster', 'SCRIPT_DIR': './FF_multi_step_multivariate', 'SCRIPT_NAME': 'FF_multi_step_multivariate.py', 'SCRIPT_PARAMS': {'--latent-dim': 5, '--hidden-layers': 2, '--batch-size': 16, '--T': 72, '--learning-rate': 0.01, '--alpha': 0.1}, 'HYPER_PARAMS': {'--latent-dim': [5, 10, 15], '--hidden-layers': [1, 2, 3], '--batch-size': [8, 16, 32], '--T': [72, 168, 336], '--learning-rate': [0.01, 0.001, 0.0001], '--alpha': [0.1, 0.001, 0]}, 'MODEL_NAME': 'ff-multistep-best'}


## Initialize workspace
Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the configuration notebook. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [17]:
# check core SDK version number
print("You are using Azure ML SDK Version: ", azureml.core.VERSION)

You are using Azure ML SDK Version:  1.0.33


In [18]:
try:
    ws = Workspace.from_config()
    print(
        "Workspace name: " + ws.name,
        "Azure region: " + ws.location,
        "Subscription id: " + ws.subscription_id,
        "Resource group: " + ws.resource_group,
        sep="\n",
    )
except:
    print(
        "Workspace not accessible. Change your parameters or create a new workspace using configuration.ipynb notebook."
    )

Workspace name: vapaunic-dnntut-ws
Azure region: westeurope
Subscription id: a61eb99b-265d-430b-91c8-911f86ae9be1
Resource group: vapaunic-aml-rg


## Create an Azure ML experiment
Let's create an Azure ML experiment. All executed runs will be recorded under this experiment in Azure.

In [20]:
exp = Experiment(workspace=ws, name=params["EXPERIMENT_NAME"])

## Upload data and scripts to default datastore 
A [datastore](https://docs.microsoft.com/azure/machine-learning/service/how-to-access-data) is a place where data can be stored that is then made accessible to a Run either by means of mounting or copying the data to the compute target. A datastore can either be backed by an Azure Blob Storage or and Azure File Share (ADLS will be supported in the future). For simple data handling, each workspace provides a default datastore that can be used, in case the data is not already in Blob Storage or File Share.

In [7]:
ds = ws.get_default_datastore()

In this next step, we will upload the training and test set into the workspace's default datastore, which we will then later be mount on an AmlCompute cluster for training.

In [8]:
ds.upload_files(
    files=["../data/GEFCom2014.zip"],
    target_path="energy",
    overwrite=True,
    show_progress=True,
)
ds.upload_files(
    files=["../common/extract_data.py", "../common/utils.py"],
    target_path="common",
    overwrite=True,
    show_progress=True,
)

Uploading ../data/GEFCom2014.zip
Uploaded ../data/GEFCom2014.zip, 1 files out of an estimated total of 1
Uploading ../common/extract_data.py
Uploading ../common/utils.py
Uploaded ../common/utils.py, 1 files out of an estimated total of 2
Uploaded ../common/extract_data.py, 2 files out of an estimated total of 2


$AZUREML_DATAREFERENCE_0417352ec970449497fba70e5db678d7

## Create or Attach existing AmlCompute
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. In this tutorial, you create `AmlCompute` as your training compute resource.

If we could not find the cluster with the given name, then we will create a new cluster here. We will create an `AmlCompute` cluster of `STANDARD_NC6` GPU VMs. This process is broken down into 3 steps:
1. create the configuration (this step is local and only takes a second)
2. create the cluster (this step will take about **20 seconds**)
3. provision the VMs to bring the cluster to the initial size (of 1 in this case). This step will take about **3-5 minutes** and is providing only sparse output in the process. Please make sure to wait until the call returns before moving to the next cell

In [22]:
# choose a name for your cluster
cluster_name = params["CLUSTER_NAME"]

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing compute target")
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_NC6", min_nodes=0, max_nodes=4
    )
    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout.
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20
    )

# use get_status() to get a detailed status for the current cluster.
print(compute_target.get_status().serialize())

Found existing compute target
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-05-08T22:38:42.061000+00:00', 'errors': None, 'creationTime': '2019-04-30T12:55:55.065120+00:00', 'modifiedTime': '2019-04-30T14:56:02.842868+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1000S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


### Azure ML concepts  
Please note the following three things in the code below:
1. The script accepts arguments using the argparse package. In this case there is one argument `--datadir` which specifies the file system folder in which the script can find the MNIST data
```
    parser = argparse.ArgumentParser()
    parser.add_argument('--datadir')
```
2. The script is accessing the Azure ML `Run` object by executing `run = Run.get_context()`. Further down the script is using the `run` to report the loss and accuracy at the end of each epoch via callback.
```
    run.log('Loss', log['loss'])
    run.log('Accuracy', log['acc'])
```
3. When running the script on Azure ML, you can write files out to a folder `./outputs` that is relative to the root directory. This folder is specially tracked by Azure ML in the sense that any files written to that folder during script execution on the remote target will be picked up by Run History; these files (known as artifacts) will be available as part of the run history record.

## Create TensorFlow estimator & add Keras
Next, we construct an `azureml.train.dnn.TensorFlow` estimator object, use the `gpucluster` as compute target, and pass the mount-point of the datastore to the training code as a parameter.
The TensorFlow estimator is providing a simple way of launching a TensorFlow training job on a compute target. It will automatically provide a docker image that has TensorFlow installed. In this case, we add `keras` package (for the Keras framework), and additional packages required for running the training script on the compute target.

We also specify `entry_script` as one of the parameters. This is the script that is going to be executed on the compute target. Please examine the script for better understanding of the training process, and the metrics that are being logged. You'll notice that for each training run, we execute `N_EXPERIMENTS` runs, in order to run the experiment with different weight initializations. We noticed that there is a good variation in obtained metrics (validation and test MAPE) across the experiments, so we use average validation MAPE over all `N_EXPERIMENTS` as the evaluation metric.

In [23]:
script_params = params["SCRIPT_PARAMS"]
script_params.update(
    {
        "--datadir": ds.path("energy").as_mount(),
        "--scriptdir": ds.path("common").as_mount(),
    }
)

est = TensorFlow(
    source_directory=params["SCRIPT_DIR"],
    script_params=script_params,
    compute_target=compute_target,
    conda_packages=["pandas", "numpy"],
    pip_packages=["keras", "matplotlib", "scikit-learn", "xlrd", "azureml-sdk"],
    entry_script=params["SCRIPT_NAME"],
    use_gpu=True,
)

framework_version is not specified, defaulting to version 1.13.


## Submit job to run
Submit the estimator to the Azure ML experiment to kick off the execution.

In [24]:
run = exp.submit(est)

In [25]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [35]:
run.get_metrics()

{'validation MAPEs': [0.13058066798581017,
  0.13081573910183378,
  0.12988554727700022,
  0.1296813256210751,
  0.12975195149580637],
 'test MAPEs': [0.23173711412391518,
  0.21802933885413006,
  0.23711992379906563,
  0.23274828174229842,
  0.23482239356161924],
 'meanValidationMAPE': 0.13014304629630513,
 'meanTestMAPE': 0.2308914104162057}

In [36]:
run.get_file_names()

['azureml-logs/55_batchai_execution.txt',
 'azureml-logs/60_control_log.txt',
 'azureml-logs/80_driver_log.txt',
 'azureml-logs/azureml.log',
 'outputs/model/bestmodel_exp3.h5',
 'outputs/model/bestmodel_exp3.json']

## Hyperparameter tuning
We have trained the model with one set of hyperparameters, now let's how we can do hyperparameter tuning by launching multiple runs on the cluster. First let's define the parameter space using random sampling.

In [39]:
hparams = dict()

for key, value in params["HYPER_PARAMS"].items():
    hparams[key] = choice(value)

ps = RandomParameterSampling(hparams)

Next, we will create a new estimator without the above parameters since they will be passed in later by Hyperdrive configuration. Note we still need to keep the `--datadir` and `--scriptdir` parameters since they are not hyperparamters we will sweep.

In [40]:
est = TensorFlow(
    source_directory=params["SCRIPT_DIR"],
    script_params={
        "--datadir": ds.path("energy").as_mount(),
        "--scriptdir": ds.path("common").as_mount(),
    },
    compute_target=compute_target,
    conda_packages=["pandas", "numpy"],
    pip_packages=["keras", "matplotlib", "scikit-learn", "xlrd", "azureml-sdk"],
    entry_script=params["SCRIPT_NAME"],
    use_gpu=True,
)

framework_version is not specified, defaulting to version 1.13.


Now we are ready to configure a run configuration object, and specify the primary metric `meanValidationMAPE` that's recorded in your training runs. We also want to tell the service that we are looking to minimize this value. We also set the number of total runs, and maximal concurrent job count, which should be up to the the number of nodes in our computer cluster.

In [44]:
hdc = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    primary_metric_name="meanValidationMAPE",
    primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
    max_total_runs=50,
    max_concurrent_runs=4,
    # max_duration_minutes=30,
)

Finally, let's launch the hyperparameter tuning job.

In [45]:
hdr = exp.submit(config=hdc)

We can use a run history widget to show the progress. Be patient as this might take a while to complete.

In [46]:
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Find and register the best model
When all the jobs finish, we can find out the one that has the lowest mean validation MAPE.

In [47]:
best_run = hdr.get_best_run_by_primary_metric()
print(best_run.get_details()["runDefinition"]["arguments"])

['--datadir', '$AZUREML_DATAREFERENCE_74e41a06add54178b4a71afca5b5aed4', '--scriptdir', '$AZUREML_DATAREFERENCE_2dff5db2615f45f890a92de56476be57', '--T', '168', '--alpha', '0', '--batch-size', '8', '--hidden-layers', '1', '--latent-dim', '10', '--learning-rate', '0.0001']


In [48]:
best_run.get_metrics()

{'validation MAPEs': [0.00237633829334906,
  0.0025241895691983196,
  0.0023787959905651706,
  0.002544753123786197,
  0.0022434681519775445],
 'test MAPEs': [0.042385739896449295,
  0.04368646865939124,
  0.04232069881219596,
  0.04289407423044128,
  0.04115717144975747],
 'meanValidationMAPE': 0.002413509025775258,
 'meanTestMAPE': 0.042488830609647046}

Now let's list the model files uploaded during the run.

In [49]:
print(best_run.get_file_names())

['azureml-logs/55_batchai_execution.txt', 'azureml-logs/60_control_log.txt', 'azureml-logs/80_driver_log.txt', 'azureml-logs/azureml.log', 'outputs/model/bestmodel_exp4.h5', 'outputs/model/bestmodel_exp4.json']


We can then register the folder (and all files in it) as a model under the workspace model registry for future deployment.

In [50]:
model = best_run.register_model(model_name=params["MODEL_NAME"], model_path='outputs/model')